In [32]:
import os
import pandas as pd
from tqdm import tqdm
import numpy as np
import datetime
from datetime import date, timedelta

In [33]:
url = 'Data/url.html'
list_file = 'Data/lista.xlsx'
data_path = 'Data'

In [51]:
def create_csv_if_not_exists(st_file):
    if not os.path.exists(st_file):
        # Crear DataFrame con los últimos 6 días en la columna 'Fecha'
        fecha_actual = pd.Timestamp.now().date()
        fechas = pd.date_range(end=fecha_actual, periods=45)
        fechas_str = fechas.strftime('%Y-%m-%d')
        df = pd.DataFrame({'Fecha': fechas_str, 'Tmax': -999, 'Tmin': -999, 'PP': -999})

        # Guardar DataFrame en un archivo CSV
        df.to_csv(st_file, index=False)
        print(f"Archivo {st_file} creado exitosamente.")

In [36]:
# Filtro de estaciones
ss = pd.read_excel(list_file)
ss = ss[ss["Dash"] == 1]

f = open(url)
rpc = pd.read_html(f.read(), header=[0, 1])[0]

# Arreglando encabezados
rpc.columns = [c[0] if c[0] == c[1] else "_".join(c) for c in rpc.columns.values]
rpc.columns.values[
    5
] = "Estacion"  # Modificar directamente los valores de la columna

# Formateando códigos de estaciones
rpc["Cod_Ant."] = rpc["Cod_Ant."].map(lambda x: "%06d" % x)
rpc["Cod."] = rpc["Cod."].map(lambda x: "%d" % x)

# Seleccionando solo los que hay en la web
idd = ss["Cod"].isin(rpc["Cod."])
ss_rpc = ss[idd]
v_rpc = ["MAX", "MIN", "PP"]
v_df = ["Tmax", "Tmin", "PP"]

In [59]:
data_dict = {}
for i in tqdm(range(len(ss_rpc)), desc="Leyendo datos de web lvera", colour="cyan"):
    cod, prv, st = ss_rpc[['Cod', 'Provincia', 'Nombre']].values[i]
    
    # Leyendo csv's con datos
    st_file = os.path.join(data_path, f'Series/{prv}/{cod}_{st}.csv')
    
    try:
        data = pd.read_csv(st_file)
    except:
        create_csv_if_not_exists(st_file)
        data = pd.read_csv(st_file)

    data['Fecha'] = pd.to_datetime(data['Fecha'])
    
    # Creando rango de fechas con última fecha de csv hasta día actual
    fe = pd.date_range(start=data['Fecha'].values[-1], end=date.today(), closed='right')

    # Generando dataframe vacío con nuevo rango de fechas
    nn = pd.DataFrame({
        'Fecha': fe,
        'Tmax': np.full(len(fe), -999),
        'Tmin': np.full(len(fe), -999),
        'PP': np.full(len(fe), -999),
    })
    
    # Uniendo data de csv's con nuevo dataframe
    df = pd.concat([data, nn], ignore_index=True)
    
    # Bucle para extraer data de web de cada columna y asignarlo al dataframe
    for a, b in zip(v_rpc, v_df):
        vc = [x for x in rpc.columns if a in x]
        ld = date.today() - timedelta(days=1)
        if a == "MIN":
            ld = date.today()
        ff = pd.date_range(end=ld, periods=len(vc))
        df.loc[df["Fecha"].isin(ff), b] = (
            rpc.loc[rpc["Cod."] == cod, vc].values[0].tolist()
        )
    
    data_dict[st] = df.tail(45)

Leyendo datos de web lvera: 100%|██████████| 22/22 [00:00<00:00, 57.44it/s]
